In [1]:
import numpy as np
from pobs.utils import get_param_from_json, save_json, load_json, load_pickle, load_hdf5, save_hdf5, append_json, get_dict_or_file
import matplotlib.pyplot as plt
import pobs

In [53]:
import pobs
from pobs.pobs import POBS
import pobs.utils as utils
import pobs.mp as mp
import pobs.modelgenerator as modelgenerator

from importlib import reload
reload(pobs)
reload(pobs.utils)
reload(pobs.modelgenerator)
reload(pobs.mp)
reload(pobs.pobs)

<module 'pobs.pobs' from '/home/hemantakumar.phurailatpam/pobs/pobs/pobs.py'>

In [4]:
def convert_to_json(
    file_dir,
    dir_ = '/home/hemantakumar.phurailatpam/pobs/processed_data/pe_results/unlensed',
    ):
    
    posterior = get_dict_or_file(file_dir)
    param_ = {}
    param_list = ['mass_1','mass_2','ra','dec', 'theta_jn','chi_eff','luminosity_distance', 'geocent_time']
    for key in param_list:
        param_[key] = np.array(posterior['posterior'][key])

    # name should include Mc, q and geocent-time
    Mc = (param_['mass_1']*param_['mass_2'])**(3/5)/(param_['mass_1']+param_['mass_2'])**(1/5)
    Mc = np.median(Mc)
    Mc = f'Mc_{Mc:.2f}'
    q = param_['mass_2']/param_['mass_1']
    q = np.median(q)
    q = f'q_{q:.2f}'
    geocent_time = np.median(param_['geocent_time'])
    geocent_time = f't_{geocent_time:.2f}'
    name = f'{dir_}/{Mc}_{q}_{geocent_time}.json'
    append_json(name, param_, replace=False);


In [5]:
# get all dir name in the folder
dir1 = '/home/hemantakumar.phurailatpam/event-catalogue/ini_automation/'
import os
files = os.listdir(dir1)
files = [f for f in files if os.path.isdir(os.path.join(dir1, f))]
# delete '__pycache__', 'outdir_test'
files = [f for f in files if f not in ['__pycache__', 'outdir_test']]

file_final = []
for file in files:
    dir2 = dir1 + file + '/result/'
    # check for hdf5 file
    files_ = os.listdir(dir2)
    files_ = [f for f in files_ if f.endswith('.hdf5')]
    if len(files_) == 0:
        print('No hdf5 file found in ', dir2)
        continue
    else:
        print('hdf5 file found in ', dir2)
        convert_to_json(dir2 + files_[0])


No hdf5 file found in  /home/hemantakumar.phurailatpam/event-catalogue/ini_automation/outdir_61_mc_27_q_0p9_time_1268203839/result/
No hdf5 file found in  /home/hemantakumar.phurailatpam/event-catalogue/ini_automation/outdir_154_mc_36_q_0p5_time_1240948205/result/
No hdf5 file found in  /home/hemantakumar.phurailatpam/event-catalogue/ini_automation/outdir_74_mc_20_q_0p9_time_1254574838/result/
hdf5 file found in  /home/hemantakumar.phurailatpam/event-catalogue/ini_automation/outdir_55_mc_49_q_0p7_time_1247724935/result/
No hdf5 file found in  /home/hemantakumar.phurailatpam/event-catalogue/ini_automation/outdir_116_mc_22_q_0p8_time_1247493549/result/
No hdf5 file found in  /home/hemantakumar.phurailatpam/event-catalogue/ini_automation/outdir_110_mc_28_q_0p7_time_1268780636/result/
No hdf5 file found in  /home/hemantakumar.phurailatpam/event-catalogue/ini_automation/outdir_160_mc_30_q_0p6_time_1243637507/result/
No hdf5 file found in  /home/hemantakumar.phurailatpam/event-catalogue/ini_

In [12]:
# get all hdf5 files in the directory
dir_ = '/home/hemantakumar.phurailatpam/pobs/processed_data/pe_results/unlensed/'
files = os.listdir(dir_)
files = [os.path.join(dir_, f) for f in files]
len(files), files[0]

(35,
 '/home/hemantakumar.phurailatpam/pobs/processed_data/pe_results/unlensed/Mc_36.16_q_0.78_t_1266619372.77.json')

In [16]:
from scipy.special import comb
import itertools

len_ = len(files)
size = 1000
num_combinations = comb(len_, 2, exact=True)
# Known number of combinations
C = size if size < num_combinations else num_combinations
print(C)
len_ = (1 + np.sqrt(1 + 8 * C)) / 2
# Define the index array
idx_all = np.arange(0, int(len_))
# # randomize idx_all
# np.random.shuffle(idx_all)

# Generate all possible two-element combinations
combination_array = np.array(list(itertools.combinations(idx_all, 2)))
idx1 = combination_array[:,0]
idx2 = combination_array[:,1]


595


In [18]:
# setting up input_arguments
input_arguments = []
size = 1000000
kde_model_type = "jax_gaussian_kde"
for i in range(len(idx1)):
    input_arguments.append([size, kde_model_type, files[idx1[i]], files[idx2[i]]])
len(idx1)

595

In [26]:
len(input_arguments)*95/60/60

15.70138888888889

In [19]:
from multiprocessing import Pool
import pobs_mp
reload(pobs_mp)

<module 'pobs_mp' from '/home/hemantakumar.phurailatpam/pobs/dev/pobs_mp.py'>

In [21]:
# save the right gaussian kde model
test = pobs.POBS(
    posterior1=None,
    posterior2=None,
    create_new=True,
    kde_model_type=kde_model_type,
    spin_zero=True,
)
# test.bayes_factor(sample_size=50)

astro_lensed is None
getting default astro_lensed data_dict from pobs module


creating a new astro_lensed model...
astro_unlensed is None
getting default astro_unlensed data_dict from pobs module
creating a new astro_unlensed1 model...
creating a new astro_unlensed2 model...
pe_prior is None
getting default pe_prior data_dict from pobs module
creating a new pe_prior model...
posterior1 is None
getting default posterior1 data_dict from pobs module
posterior2 is None
getting default posterior2 data_dict from pobs module
creating a new posterior1 model...
creating a new posterior2 model...
creating a new posterior_combine model...


(np.float64(10678464340374.756), np.float64(13.02850880171368))

In [23]:
# size=10000, time=50.3
# size=1000000, time=1m 35s=95s
test.bayes_factor(sample_size=1000000)

(np.float64(992432579284.5095), np.float64(11.996701012725568))

In [53]:
# non-jax: sample_size=50, time=2m 18.7s, num_pair=28, 
# jax: sample_size=50, time=2m 18.7s, num_pair=28, 
results = []
npool = 4
# with Pool(processes=npool) as pool:
#     for result in pool.map(pobs_mp.pobs_mp, input_arguments):
#         results.append(list(result))
# use for loop
for i in range(len(input_arguments)):
    results.append(pobs_mp.pobs_mp(input_arguments[i]))

In [54]:
results

[(0.009704223780248813, -2.013039197157287),
 (0.04315980474680531, -1.3649205287151889),
 (0.000869091987074163, -3.060934254202128),
 (1.0003489967538141e-09, -8.999848459077723),
 (0.6311608717088917, -0.199859932687805),
 (0.004256438003963368, -2.37095368782845),
 (2.3230836826000918e-12, -11.633935145672485),
 (0.001075988965104589, -2.96819218259041),
 (2.7073082129843652e-17, -16.567462299242898),
 (0.0003273649269909164, -3.4849678515894356)]

In [55]:
import os
import numpy as np
import pobs
from scipy.special import comb
import itertools
import pobs_mp
from pobs.utils import append_json


# get all hdf5 files in the directory
dir_ = '/home/hemantakumar.phurailatpam/pobs/processed_data/pe_results/unlensed/'
files = os.listdir(dir_)
files = [os.path.join(dir_, f) for f in files]

len_ = len(files)
size = 1000
num_combinations = comb(len_, 2, exact=True)
# Known number of combinations
C = size if size < num_combinations else num_combinations
print(C)
len_ = (1 + np.sqrt(1 + 8 * C)) / 2
# Define the index array
idx_all = np.arange(0, int(len_))
# # randomize idx_all
# np.random.shuffle(idx_all)

# Generate all possible two-element combinations
combination_array = np.array(list(itertools.combinations(idx_all, 2)))
idx1 = combination_array[:,0]
idx2 = combination_array[:,1]

# setting up input_arguments
input_arguments = []
size = 100000
kde_model_type = "jax_gaussian_kde"
for i in range(len(idx1)):
    input_arguments.append([size, kde_model_type, files[idx1[i]], files[idx2[i]]])

save_file_name = 'unlensed_pobs_results.json'

for i in range(len(input_arguments)):
    result = pobs_mp.pobs_mp(input_arguments[i])
    dict_ = {
        'bayes_factor': [result[0]],
        'log10_bayes_factor': [result[1]],
    }
    append_json(save_file_name, dict_, replace=False)

: 

In [37]:
from pobs.utils import get_dict_or_file
data_ = get_dict_or_file(input_arguments[0][2])

In [38]:
data_.keys()

dict_keys(['mass_1', 'mass_2', 'ra', 'dec', 'theta_jn', 'chi_eff', 'luminosity_distance', 'geocent_time'])

In [48]:
import pobs

test = pobs.POBS(
        posterior1=input_arguments[0][2],
        posterior2=input_arguments[0][3],
        # create_new=True,
        kde_model_type=kde_model_type,
        spin_zero=True,
        npool=1,
    )

astro_lensed is None
getting default astro_lensed data_dict from pobs module
creating a new astro_lensed model...
astro_unlensed is None
getting default astro_unlensed data_dict from pobs module
creating a new astro_unlensed1 model...
creating a new astro_unlensed2 model...
pe_prior is None
getting default pe_prior data_dict from pobs module
creating a new pe_prior model...
dict_keys(['mass_1', 'mass_2', 'ra', 'dec', 'theta_jn', 'chi_eff', 'luminosity_distance', 'geocent_time'])


ValueError: mass_1 is not present in the posterior2